In [25]:
import mlrun
from mlrun.errors import MLRunRuntimeError
from mlrun.runtimes import RunError
import os

In [2]:
project_name1 = "test-git-flow"
project = mlrun.get_or_create_project(project_name1, "./")

> 2023-12-05 12:32:35,899 [info] Project loaded successfully: {'project_name': 'test-git-flow'}


## Check set_source does change the project spec

In [3]:
source_path = "git://github.com/mlrun/project-demo#main"
project.set_source(source_path, pull_at_runtime=True)
assert project.spec.source==source_path

> 2023-12-05 12:32:41,093 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/User/automations/git-flow'}


## Check the auto_build parameter + check that pull_at_runtime=True works
(There's no way of really checking if pull_at_runtime works like it should during automation tests.. So I'll check that it works with both values)

In [4]:
handler_name = "gen_iris.iris_generator"
func1_name = "func1-name"
func1 = project.set_function(handler=handler_name, name=func1_name, image="mlrun/mlrun", kind="job", with_repo=True)

### Run the function with auto_build=False
#### We expect to receive an MLRunRuntimeError

In [5]:
try:
    func1_run = project.run_function(func1_name, local=False, auto_build=False)
except Exception as e:
    assert isinstance(e, MLRunRuntimeError)

> 2023-12-05 12:32:45,280 [info] Storing function: {'name': 'func1-name-iris-generator', 'uid': '61eda4450b2a4c37a0e9508e6d201f81', 'db': 'http://mlrun-api:8080'}
> 2023-12-05 12:32:45,595 [info] Job is running in the background, pod: func1-name-iris-generator-xfdjq


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-git-flow-amite,...6d201f81,0,Dec 05 12:32:49,completed,func1-name-iris-generator,v3io_user=amitekind=jobowner=amitemlrun/client_version=1.5.2-rc1mlrun/client_python_version=3.9.16host=func1-name-iris-generator-xfdjq,,,,iris_dataset


> 2023-12-05 12:32:58,744 [info] Run execution finished: {'status': 'completed', 'name': 'func1-name-iris-generator'}


### Now we set auto_build=True
#### We expect the function to run successfully

In [6]:
func1_run = project.run_function(func1_name, local=False, auto_build=True)
assert isinstance(func1_run, mlrun.model.RunObject)
assert func1_run.spec.handler==handler_name

> 2023-12-05 12:33:33,308 [info] Storing function: {'name': 'func1-name-iris-generator', 'uid': '570c0a85f1e34d8298d58c2dc0458e82', 'db': 'http://mlrun-api:8080'}
> 2023-12-05 12:33:35,684 [info] Job is running in the background, pod: func1-name-iris-generator-p72h7


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-git-flow-amite,...c0458e82,0,Dec 05 12:33:39,completed,func1-name-iris-generator,v3io_user=amitekind=jobowner=amitemlrun/client_version=1.5.2-rc1mlrun/client_python_version=3.9.16host=func1-name-iris-generator-p72h7,,,,iris_dataset


> 2023-12-05 12:33:48,819 [info] Run execution finished: {'status': 'completed', 'name': 'func1-name-iris-generator'}


## Check the with_repo parameter + check that pull_at_runtime=False works
### We will create a new project to check pull_at_runtime=False


In [7]:
project_name2 = "test-git-flow2"
project = mlrun.get_or_create_project(project_name2, "./")

> 2023-12-05 12:33:51,196 [info] Loading project from path: {'project_name': 'test-git-flow2', 'path': './'}
> 2023-12-05 12:34:06,850 [info] Project loaded successfully: {'project_name': 'test-git-flow2', 'path': './', 'stored_in_db': True}


In [8]:
project.set_source("git://github.com/mlrun/project-demo#main", pull_at_runtime=False)

> 2023-12-05 12:34:06,875 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/User/automations/git-flow'}


#### First we try with_repo=False. Since source is from github we expect an error

In [9]:
handler_name = "gen_iris.iris_generator"
func2_name = "func2-name"
func2 = project.set_function(handler=handler_name, name=func2_name, image="mlrun/mlrun", kind="job", with_repo=False)

In [10]:
try:
    func2_run = project.run_function(func2_name, local=False, auto_build=True)
except Exception as e:
    assert isinstance(e, RunError)

> 2023-12-05 12:34:06,924 [info] Storing function: {'name': 'func2-name-iris-generator', 'uid': '1966718e313740cc9a94c58a8522f635', 'db': 'http://mlrun-api:8080'}
> 2023-12-05 12:34:07,177 [info] Job is running in the background, pod: func2-name-iris-generator-cvcjv


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-git-flow2-amite,...8522f635,0,Dec 05 12:34:10,error,func2-name-iris-generator,v3io_user=amitekind=jobowner=amitemlrun/client_version=1.5.2-rc1mlrun/client_python_version=3.9.16host=func2-name-iris-generator-cvcjv,,,,


> 2023-12-05 12:34:20,284 [info] Run execution finished: {'status': 'error', 'name': 'func2-name-iris-generator'}
> 2023-12-05 12:34:20,285 [error] Run did not finish successfully: {'state': 'error', 'status': {'state': 'error', 'error': "failed on pre-loading / post-running of the function, caused by: state/function init failed, handler gen_iris.iris_generator not found, caused by: No module named 'gen_iris'", 'artifacts': [], 'start_time': '2023-12-05T12:34:10.442942+00:00', 'last_update': '2023-12-05T12:34:10.549558+00:00'}}


#### Now we set with_repo=True - we expect to succeed

In [11]:
func2 = project.set_function(handler=handler_name, name=func2_name, image="mlrun/mlrun", kind="job", with_repo=True)

In [12]:
func2_run = project.run_function(func2_name, local=False, auto_build=True)
assert isinstance(func2_run, mlrun.model.RunObject)
assert func2_run.spec.handler==handler_name

> 2023-12-05 12:34:38,382 [info] Function is not deployed and auto_build flag is set, starting deploy...
> 2023-12-05 12:34:38,711 [info] Started building image: .mlrun/func-test-git-flow2-amite-func2-name:latest
> 2023-12-05 12:35:40,178 [info] Storing function: {'name': 'func2-name-iris-generator', 'uid': '51fb26282d544436a63c36011ac0c1ea', 'db': 'http://mlrun-api:8080'}
> 2023-12-05 12:35:43,072 [info] Job is running in the background, pod: func2-name-iris-generator-bnq6t
> 2023-12-05 12:35:48,004 [info] saving iris dataframe to v3io:///projects/test-git-flow2-amite/artifacts
> 2023-12-05 12:35:48,229 [info] Run execution finished: {'status': 'completed', 'name': 'func2-name-iris-generator'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-git-flow2-amite,...1ac0c1ea,0,Dec 05 12:35:47,completed,func2-name-iris-generator,v3io_user=amitekind=jobowner=amitemlrun/client_version=1.5.2-rc1mlrun/client_python_version=3.9.16host=func2-name-iris-generator-bnq6t,,,,iris_dataset


> 2023-12-05 12:35:51,784 [info] Run execution finished: {'status': 'completed', 'name': 'func2-name-iris-generator'}


## Check the 'create_remote' function

In [13]:
project_name3 = "test-git-flow3"
project = mlrun.get_or_create_project(project_name3, "./", init_git=True)


> 2023-12-05 12:35:56,753 [info] Loading project from path: {'project_name': 'test-git-flow3', 'path': './'}
> 2023-12-05 12:36:12,827 [info] Project loaded successfully: {'project_name': 'test-git-flow3', 'path': './', 'stored_in_db': True}


In [14]:
source_path = 'git://github.com/mlrun/project-demo'
project.create_remote(url=source_path,name='test-git-flow-remote',branch='main')
assert project.spec.source==source_path+"#refs/heads/master"
assert project.spec.origin_url==source_path+"#main"

### Check the 'load_project' function

In [15]:
source_path = 'git://github.com/mlrun/project-demo'
folder_name = "loaded_project"
project = mlrun.load_project(folder_name, source_path, clone=True)


In [16]:
folder_name = "loaded_project"
created_files = ["gen_iris.py", "kflow.py", "LICENSE", "newflow.py", "prep_data.py", "project.yaml", "README.md"]

assert os.path.exists(folder_name), f"The file '{folder_name}' does not exist."
for file_name in created_files:
    file_path = folder_name+"/"+file_name
    assert os.path.exists(file_path), f"The file '{file_path}' does not exist."

## Check the 'export' function

In [17]:
export_file_name = "export-file.yaml"
project.export(export_file_name)
assert os.path.exists(export_file_name), f"The file '{export_file_name}' does not exist."


## Delete all the resources

In [23]:
run_db = mlrun.get_run_db()

In [24]:
run_db.delete_project(project_name1, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name2, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name3, mlrun.common.schemas.constants.DeletionStrategy.cascade)

> 2023-12-05 12:46:26,128 [info] Project is being deleted: {'project_name': 'test-git-flow-amite'}
> 2023-12-05 12:46:50,825 [info] Project deleted: {'project_name': 'test-git-flow-amite'}
> 2023-12-05 12:46:51,106 [info] Project is being deleted: {'project_name': 'test-git-flow2-amite'}
> 2023-12-05 12:47:07,956 [info] Project deleted: {'project_name': 'test-git-flow2-amite'}
> 2023-12-05 12:47:08,249 [info] Project is being deleted: {'project_name': 'test-git-flow3-amite'}
> 2023-12-05 12:47:26,468 [info] Project deleted: {'project_name': 'test-git-flow3-amite'}
